# E15. seq2seq 단어 단위 번역기 만들기

In [1]:
import pandas as pd
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np

1 Physical GPUs, 1 Logical GPUs


In [2]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking
from tensorflow.keras.models import Model
import re
print('⏳')

⏳


__훈련 데이터 샘플 33000개만 사용하기__

In [3]:
import os
file_path = os.getenv('HOME')+'/aiffel/e/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')

lines = lines[['eng','fra']][:33000]
print('전체 샘플의 수 :',len(lines))
lines.sample(5) #샘플 5개 출력

전체 샘플의 수 : 33000


,eng,fra
14626,I can't save you.,Je ne peux pas te sauver.
27937,You're a funny man.,Vous êtes un drôle de type.
14712,I don't hate you.,Je ne te hais point.
20209,It depends on you.,Cela dépend de vous.
7262,Are you hiring?,Recrutez-vous ?


## Step 1. 정제, 정규화, 전처리 (영어, 프랑스어 모두!)

In [4]:
def preprocess_sentence(sentence):
    # 2. 소문자로 바꾸기
    sentence = sentence.lower().strip()
    
    # 1. 구두점을 단어와 분리
    sentence = re.sub(r"([.,!?])",r" \1 ", sentence)
    sentence = re.sub(r"[']", r"", sentence)
    
    sentence = sentence.strip().split()
    return sentence

In [5]:
sentence = "Hi my name's Slim Shady."
sentence = preprocess_sentence(sentence)
print(sentence)

['hi', 'my', 'names', 'slim', 'shady', '.']


In [6]:
lines.describe()

,eng,fra
count,33000,33000
unique,20971,30180
top,You're the teacher.,Merci bien.
freq,26,9


## Step 2. 디코더의 문장에 시작 토큰과 종료 토큰을 넣어주세요.

In [7]:
lines.eng = lines.eng.apply(lambda x: preprocess_sentence(x))
lines.fra = lines.fra.apply(lambda x: preprocess_sentence(x))
lines.sample(5)

,eng,fra
3346,"[come, join, us, .]","[viens, te, joindre, à, nous, !]"
30451,"[im, a, beginner, ,, too, .]","[je, suis, également, un, débutant, .]"
3507,"[he, let, us, go, .]","[il, nous, a, laissés, partir, .]"
27304,"[what, a, strange, dog, !]","[quel, chien, étrange, !]"
20448,"[let, me, pay, for, it, .]","[laissez-moi, le, payer, .]"


In [8]:
sos_token = '<sos>'
eos_token = '<eos>'

lines.fra = lines.fra.apply(lambda x: ['<sos>'] + x + ['<eos>'])
lines.sample(5)

,eng,fra
25939,"[she, looks, lonesome, .]","[<sos>, elle, a, lair, solitaire, ., <eos>]"
12568,"[they, can, manage, .]","[<sos>, ils, peuvent, gérer, ., <eos>]"
25679,"[my, battery, is, dead, .]","[<sos>, ma, batterie, est, vide, ., <eos>]"
31854,"[the, crops, need, rain, .]","[<sos>, les, cultures, ont, besoin, de, pluie,..."
23565,"[he, banged, his, knee, .]","[<sos>, il, sest, cogné, au, genou, ., <eos>]"


## Step 3. 케라스의 토크나이저로 텍스트를 숫자로 바꿔보세요.
---
tokenizer.texts_to_sequences()를 사용하여 모든 샘플에 대해서 정수 시퀀스로 변환

In [9]:
eng_tokenizer = Tokenizer(char_level=False) # 단어 단위 Tokenizer
eng_tokenizer.fit_on_texts(lines.eng)       # 33000개 eng 행에 각각 토큰화
input_text = eng_tokenizer.texts_to_sequences(lines.eng)  # 단어 -> 숫자값 인덱스 변환 저장
input_text[:3]

[[28, 1], [1160, 1], [1160, 1]]

In [10]:
fra_tokenizer = Tokenizer(char_level=False)
fra_tokenizer.fit_on_texts(lines.fra)
target_text = fra_tokenizer.texts_to_sequences(lines.fra)
target_text[:3]

[[1, 77, 7, 2], [1, 1087, 7, 2], [1, 1087, 3, 2]]

In [11]:
eng_vocab_size = len(eng_tokenizer.word_index) + 1
fra_vocab_size = len(fra_tokenizer.word_index) + 1
print("영어 단어장 크기:", eng_vocab_size, "\n불어 단어장 크기", fra_vocab_size)

영어 단어장 크기: 4786 
불어 단어장 크기 9839


In [12]:
max_eng_seq_len = max([len(line) for line in input_text])
max_fra_seq_len = max([len(line) for line in target_text])
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 15


__전체 통계량__

In [13]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 4786
프랑스어 단어장의 크기 : 9839
영어 시퀀스의 최대 길이 8
프랑스어 시퀀스의 최대 길이 15


__시작, 종료 토큰 각각 제거__

In [14]:
lines.fra.sample(3)

16601      [<sos>, ils, ont, tous, arrêté, ., <eos>]
8137           [<sos>, je, vis, en, ville, ., <eos>]
29010    [<sos>, il, est, fort, au, rugby, ., <eos>]
Name: fra, dtype: object

In [15]:
sos_token = '<sos>'
eos_token = '<eos>'

encoder_input = input_text
decoder_input = [[word for word in line if word != fra_tokenizer.word_index[eos_token]] for line in target_text]
decoder_target = [[word for word in line if word != fra_tokenizer.word_index[sos_token]] for line in target_text]

In [16]:
print(decoder_input[:3])
print(decoder_target[:3])

[[1, 77, 7], [1, 1087, 7], [1, 1087, 3]]
[[77, 7, 2], [1087, 7, 2], [1087, 3, 2]]


In [17]:
# padding
encoder_input = pad_sequences(encoder_input, maxlen = max_eng_seq_len, padding="post")
decoder_input = pad_sequences(decoder_input, maxlen = max_fra_seq_len, padding="post")
decoder_target = pad_sequences(decoder_target, maxlen = max_fra_seq_len, padding="post")
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 8)
프랑스어 입력데이터의 크기(shape) : (33000, 15)
프랑스어 출력데이터의 크기(shape) : (33000, 15)


__shuffle__

In [18]:
idx = np.arange(encoder_input.shape[0])
np.random.shuffle(idx)

encoder_input = encoder_input[idx]
decoder_input = decoder_input[idx]
decoder_target = decoder_target[idx]

In [19]:
print(encoder_input[0])
print(idx)

[ 20   4  37 297   3   0   0   0]
[13912 25462 26631 ... 32897 22264 13212]


In [20]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]


In [21]:
print('영어 학습데이터의 크기(shape) :',np.shape(encoder_input_train))
print('프랑스어 학습 입력데이터의 크기(shape) :',np.shape(decoder_input_train))
print('프랑스어 학습 출력데이터의 크기(shape) :',np.shape(decoder_target_train))

print('영어 검증데이터의 크기(shape) :',np.shape(encoder_input_test))
print('프랑스어 검증 입력데이터의 크기(shape) :',np.shape(decoder_input_test))
print('프랑스어 검증 출력데이터의 크기(shape) :',np.shape(decoder_target_test))

영어 학습데이터의 크기(shape) : (30000, 8)
프랑스어 학습 입력데이터의 크기(shape) : (30000, 15)
프랑스어 학습 출력데이터의 크기(shape) : (30000, 15)
영어 검증데이터의 크기(shape) : (3000, 8)
프랑스어 검증 입력데이터의 크기(shape) : (3000, 15)
프랑스어 검증 출력데이터의 크기(shape) : (3000, 15)


## Step 4. 임베딩 층(Embedding layer) 사용하기
---
이번에는 입력이 되는 각 단어를 임베딩 층을 사용하여 벡터화하겠습니다.

임베딩 층을 사용하는 방법과 그 설명에 대해서는 아래의 링크의

__1. 케라스 임베딩 층(Keras Embedding layer)__를 참고하세요.

[위키독스](https://wikidocs.net/33793)

실제 번역기 구현을 위해서 사용할 수 있는 인코더 코드의 예시는 다음과 같습니다.

이를 통해서 인코더와 디코더의 임베딩 층을 각각 구현해보세요.

from tensorflow.keras.layers import Input, mbedding, Masking

```python
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(단어장의 크기, 임베딩 벡터의 차원)(encoder_inputs)
encoder_lstm = LSTM(hidden state의 크기, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
```

주의할 점은 인코더와 디코더의 임베딩 층은 서로 다른 임베딩 층을 사용해야 하지만,

디코더의 훈련 과정과 테스트 과정(예측 과정)에서의 임베딩 층은 동일해야 합니다!

__인코더 임베딩__

In [22]:
output_size = 32

In [23]:
encoder_inputs = Input(shape=(None,))

enc_emb = Embedding(eng_vocab_size, output_size, input_length=max_eng_seq_len)(encoder_inputs)
# padding값은 연산하지 않는다
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(units=output_size, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [24]:
encoder_inputs.shape

TensorShape([None, None])

__디코더 임베딩__

In [25]:
decoder_inputs = Input(shape=(None, ))
dec_emb = Embedding(fra_vocab_size, output_size)(decoder_inputs)

print(decoder_inputs.shape)
print(dec_emb.shape, "\n")

dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(units=output_size, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

print(dec_masking.shape)

decoder_softmax_layer = Dense(fra_vocab_size, activation="softmax")
decoder_outputs = decoder_softmax_layer(decoder_outputs)

decoder_outputs.shape

(None, None)
(None, None, 32) 

(None, None, 32)


TensorShape([None, None, 9839])

## Step 5. 모델 구현하기
---
글자 단위 번역기에서 구현한 모델을 참고로 단어 단위 번역기의 모델을 완성시켜보세요!

In [26]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["acc"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 32)     153152      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 32)     314848      input_2[0][0]                    
______________________________________________________________________________________________

__학습__

In [27]:
model.fit([encoder_input_train, decoder_input_train], decoder_target_train, \
         validation_data = ([encoder_input_test, decoder_input_test], decoder_target_test),
         batch_size=128, epochs=40)

Epoch 1/40
235/235 [==============================] - 10s 43ms/step - loss: 4.0328 - acc: 0.6055 - val_loss: 2.2377 - val_acc: 0.6071
Epoch 2/40
235/235 [==============================] - 9s 40ms/step - loss: 1.9729 - acc: 0.6766 - val_loss: 1.8125 - val_acc: 0.7363
Epoch 3/40
235/235 [==============================] - 9s 37ms/step - loss: 1.7049 - acc: 0.7414 - val_loss: 1.6582 - val_acc: 0.7484
Epoch 4/40
235/235 [==============================] - 9s 37ms/step - loss: 1.5846 - acc: 0.7589 - val_loss: 1.5727 - val_acc: 0.7652
Epoch 5/40
235/235 [==============================] - 9s 37ms/step - loss: 1.5076 - acc: 0.7691 - val_loss: 1.5155 - val_acc: 0.7727
Epoch 6/40
235/235 [==============================] - 9s 37ms/step - loss: 1.4524 - acc: 0.7772 - val_loss: 1.4671 - val_acc: 0.7805
Epoch 7/40
235/235 [==============================] - 9s 38ms/step - loss: 1.4059 - acc: 0.7841 - val_loss: 1.4240 - val_acc: 0.7872
Epoch 8/40
235/235 [==============================] - 9s 38ms/step -

__test__

In [28]:
enc_model = Model(inputs=encoder_inputs, outputs = encoder_states)
enc_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 32)          153152    
_________________________________________________________________
masking (Masking)            (None, None, 32)          0         
_________________________________________________________________
lstm (LSTM)                  [(None, 32), (None, 32),  8320      
Total params: 161,472
Trainable params: 161,472
Non-trainable params: 0
_________________________________________________________________


In [29]:
# decoder
decoder_state_input_h = Input(shape=(output_size, ))
decoder_state_input_c = Input(shape=(output_size, ))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb_test = Embedding(fra_vocab_size, output_size)(decoder_inputs)
decoder_outputs_test, state_h_test, state_c_test = decoder_lstm(dec_emb_test, initial_state = decoder_states_inputs)
decoder_states_test = [state_h_test, state_c_test]

decoder_outputs_test = decoder_softmax_layer(decoder_outputs_test)

In [30]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs_test] + decoder_states_test)
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 32)     314848      input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 32)]         0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 32)]         0                                            
____________________________________________________________________________________________

In [31]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [32]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = enc_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = fra2idx["<sos>"]

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' ' + sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<eos>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [33]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 반환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if (i!=0):
            temp = temp + idx2eng[i] + ' '
    return temp

In [34]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<sos>']) and i!=fra2idx['<eos>']):
            temp = temp + idx2fra[i] + ' '
    return temp

## Step 6. 모델 평가하기
---
단어 단위 번역기에 대해서 훈련 데이터의 샘플과 테스트 데이터의 샘플에 대해서 번역 문장을 만들어보고 정답 문장과 번역 문장을 비교해보세요.

In [35]:
import numpy as np
for seq_index in [1, 100, 155, 303, 888, 1012]:
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.eng[seq_index])
    print('정답 문장:', lines.fra[seq_index][1:len(lines.fra[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: ['hi', '.']
정답 문장: ['salut', '!']
번역기가 번역한 문장:  ce pas . . . . 
-----------------------------------
입력 문장: ['call', 'us', '.']
정답 문장: ['appelez-nous', '!']
번역기가 번역한 문장:  la en . . . . 
-----------------------------------
입력 문장: ['i', 'dozed', '.']
정답 문장: ['je', 'me', 'suis', 'assoupie', '.']
번역기가 번역한 문장:  les les . . . 
-----------------------------------
입력 문장: ['how', 'rude', '!']
정답 문장: ['quelle', 'grossièreté', '!']
번역기가 번역한 문장:  ne à . . . . . 
-----------------------------------
입력 문장: ['of', 'course', '!']
정답 문장: ['pardi', '!']
번역기가 번역한 문장:  la . . . . . . 
-----------------------------------
입력 문장: ['we', 'did', 'it', '.']
정답 문장: ['nous', 'lavons', 'fait', '.']
번역기가 번역한 문장:  vous de de . . 


### 결론
- 번역기가 전혀 제대로 작동하고 있지 않다
- 정규표현식과 파라미터를 바꿔도 마찬가지이다
- nlp에 대해 좀 더 집중적으로 공부해보고 나서 다시 도전해봐야겠다.